## Overview

The goal of this notebook is to gather data from the National Transit Database (NTD) and parse it into something usable for our transit carbon calculations. There are two datasets we need to import and parse, one being the [NTD Annual Data - Fuel and Energy](https://data.transportation.gov/Public-Transit/2022-NTD-Annual-Data-Fuel-and-Energy/8ehq-7his/data) set, and the other being the [TS2.2 - Service Data and Operating Expenses Time Series by System](https://www.transit.dot.gov/ntd/data-product/ts22-service-data-and-operating-expenses-time-series-system-0) set. The former gives us data on fuel types used by certain transit agencies, while the latter gives us the passenger miles traveled (PMT) of each agency. Once the notebook has ran, all of the parsed data goes into `revised-ntd-json-data.json`. The data format is as follows...

<details>
<summary>Data Format</summary>

```json
{
    "meta": {
        // Meta data, column names, etc...
    },
    "data": {
        // uace_code: [ array of dictionaries/all rows that pertain to the uace_code ]
    },
    "pmt": {
        // uace_code: total_pmt_value
    }
}
```

</details>

#### To Maintain

To update the data each year, update the two url variables below with their most recent download link.

In [153]:
# NTD Annual Data - Fuel and Energy: https://data.transportation.gov/Public-Transit/2022-NTD-Annual-Data-Fuel-and-Energy/8ehq-7his/data
ntd_fuel_energy_url = "https://data.transportation.gov/api/views/8ehq-7his/rows.json?accessType=DOWNLOAD"

ntd_fuel_energy_urls = [
    {
        # https://www.transit.dot.gov/ntd/data-product/2022-fuel-and-energy | CSV file
        "year": "2022",
        "file_type": "csv",
        "url": "https://data.transportation.gov/api/views/8ehq-7his/rows.csv?date=20231027&accessType=DOWNLOAD&bom=true&format=true"
    }, 
    {
        # https://www.transit.dot.gov/ntd/data-product/2021-fuel-and-energy | xlsx file
        "year": "2021",
        "file_type": "xlsx",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2023-12/2021%20Fuel%20and%20Energy_1-1_0.xlsx"
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2020-fuel-and-energy | xlsx file, Data is organized differently
        "year": "2020",
        "file_type": "xlsx",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2023-12/2020-Fuel%20and%20Energy_1-1_1.xlsx",
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2019-fuel-and-energy | ZIP file then xlsm, Data is organized differently
        "year": "2019",
        "file_type": "zip",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Fuel%20and%20Energy.zip",
    }, 
    {
        # https://www.transit.dot.gov/ntd/data-product/2018-fuel-and-energy | xlsm, Data is organized differently
        "year": "2018",
        "file_type": "xlsm",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Fuel%20and%20Energy_3.xlsm"
    }
]

ntd_service_pmt_urls = [
    {
        # https://www.transit.dot.gov/ntd/data-product/2022-service
        "year": "2022",
        "file_type": "csv",
        "url": "https://data.transportation.gov/api/views/4fir-qbim/rows.csv?date=20231102&accessType=DOWNLOAD&bom=true&format=true"
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2021-service | Annual Service Data by Mode
        "year": "2021",
        "file_type": "xlsx",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2022-10/2021%20Service_static.xlsx"
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2020-service | Annual Service Data by Mode
        "year": "2020",
        "file_type": "zip",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/2020-Service.zip"
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2019-service | Annual Service Data by Mode
        "year": "2019",
        "file_type": "zip",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Service.zip"
    },
    {
        # https://www.transit.dot.gov/ntd/data-product/2018-service | Annual Service Data by Mode
        "year": "2018",
        "file_type": "xlsm",
        "url": "https://www.transit.dot.gov/sites/fta.dot.gov/files/Service_4.xlsm"
    },

]

### Download the NTD Fuel and Energy + NTD Service data

In [171]:
import json
import requests
import pandas as pd
import zipfile
import os
from collections import defaultdict

# Download file helper function
def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

# Download all NTD Fuel and Energy Data 
for url_entry in ntd_fuel_energy_urls:
    ntd_fuel_energy_name = url_entry["year"] + "-fuel-energy." + url_entry["file_type"]
    download_file(url_entry["url"], ntd_fuel_energy_name)

# Download all NTD Service Data 
for url_entry in ntd_service_pmt_urls:
    ntd_service_pmt_name = url_entry["year"] + "-service-pmt." + url_entry["file_type"]
    download_file(url_entry["url"], ntd_service_pmt_name)

### Unify the NTD Fuel and Energy data into one JSON file

All of the NTD data between the years is split up into different file structures and formats, so we need to unify that into one cohesive JSON file. We will also need to refactor all of the data a bit by converting all the rows, which are represented by arrays, into dictionaries, removing extraneous values, and adding additional fields where necessary.

In [172]:
ntd_fuel_energy_data = {}
ntd_to_uace = {}

def refactor_fuel_energy_data(url_entry):
    '''
    Take all of the NTD Fuel and Energy data and do the following
    1) Load in the data from the file
    2) Convert the rows into dictionaries
    3) Group all the rows by UACE code
    4) Write all the data to a single file
    '''
    # Load in data
    df = load_dataframe(url_entry, "fuel-energy", "Fuel and Energy")
    # Ensure there are no NaN values
    df = df.fillna(0)
    # Convert all rows that are arrays into dictionaries
    converted_rows = convert_arrays_dictionary(df, url_entry["year"])
    # Group all rows by UACE code
    aggregate_data = group_by_uace(converted_rows)
    # Add to data
    ntd_fuel_energy_data[url_entry["year"]] = [aggregate_data]
    print("Service data year " + url_entry["year"] + " is finished!")

def load_dataframe(url_entry, name, sheet_name):
    '''
    Open a dataframe for the file we just downloaded based upon what file type it was
    '''
    df = pd.DataFrame()
    file_name = url_entry["year"] + "-" + name + "." + url_entry["file_type"]
    if url_entry["file_type"] == "csv":
        df = pd.read_csv(file_name)
    elif url_entry["file_type"] == "xlsx" or url_entry["file_type"] == "xlsm":
        df = pd.read_excel(file_name, sheet_name=sheet_name)
    elif url_entry["file_type"] == "zip":
        with zipfile.ZipFile(file_name, "r") as zip_ref:
            with zip_ref.open(zip_ref.namelist()[0]) as xlsm_file:
                df = pd.read_excel(xlsm_file, sheet_name=sheet_name)
    return df
    
def convert_arrays_dictionary(df, year):
    '''
    Takes in a dataframe and converts all of the rows into a dictionary.
    '''
    converted_rows = []
    # Previous years had a different column name structure than 2022
    if int(year) < 2022:
        # Map column names to each value in the row, and remove "Questionable" fields
        for _, row in df.iterrows():
            temp_row = row.to_dict()
            converted_row = {}
            miles = False
            for k, v in temp_row.items():
                if k == "Miles Traveled by Vehicles Fueled by:" or k == "Miles per Gallon/KwH:":
                    # Once we get to the start or end of the miles columns, note it
                    miles = not miles
                    continue
                if miles:
                    # If at the miles column, add it to the name
                    k = k + " (miles)"
                if "Questionable" not in str(k):
                    # Only add k,v back if it is not a "Questionable" field
                    converted_row[k] = v
            converted_row["UACE Code"] = ntd_to_uace.get(converted_row["NTD ID"], -1)
            converted_rows.append(converted_row)
    else:
        converted_rows = [{k: v for k, v in row.to_dict().items() if "Questionable" not in k} for _, row in df.iterrows()]
    
    return converted_rows

def group_by_uace(converted_rows):
    '''
    Organize the data into UACE codes.
    Example data:
    {
        "uace_code": [
            {
                "field1": 1,
                ...
            },
            {
                "field1": 1,
                ...
            }
        ]
    } 
    '''
    aggregate_data = defaultdict(list)
    for row in converted_rows:
        code = row["UACE Code"]
        aggregate_data[code].append(row)
    return aggregate_data

def map_ntd_to_uace():
    '''
    Use 2022 data to Create a mapping between NTD ids and UACE codes because older versions of the data
    2018-2021 don't have an UACE field.
    '''
    df = pd.read_csv("2022-fuel-energy.csv")
    for _, row in df.iterrows():
        temp = row.to_dict()
        ntd_to_uace[temp["NTD ID"]] = temp["UACE Code"]
        
def delete_old_files(file_name, urls):
    '''
    Delete all old NTD Fuel and Energy data files
    '''
    for url_entry in urls:
        file_path = url_entry["year"] + "-" + file_name + "." + url_entry["file_type"]
        try:
            os.remove(file_path)
            print(f"The file {file_path} has been removed successfully.")
        except FileNotFoundError:
            print(f"The file {file_path} does not exist.")
        except Exception as e:
            print(f"An error occurred: {e}")

# Create NTD to UACE mapping
map_ntd_to_uace()
            
# For each year of data, refactor it
for url_entry in ntd_fuel_energy_urls:
    refactor_fuel_energy_data(url_entry)

# Add all the data to a file
ntd_fuel_energy_data_json = json.dumps(ntd_fuel_energy_data, indent=2)
with open("ntd_fuel_energy.json", 'w') as file:
    file.write(ntd_fuel_energy_data_json)

# Remove the old files
delete_old_files("fuel-energy", ntd_fuel_energy_urls)

Service data year 2022 is finished!
Service data year 2021 is finished!
Service data year 2020 is finished!
Service data year 2019 is finished!
Service data year 2018 is finished!
The file 2022-fuel-energy.csv has been removed successfully.
The file 2021-fuel-energy.xlsx has been removed successfully.
The file 2020-fuel-energy.xlsx has been removed successfully.
The file 2019-fuel-energy.zip has been removed successfully.
The file 2018-fuel-energy.xlsm has been removed successfully.


### PMT Data

In order to properly assess the amount of carbon emitted when a person uses public transit, it is important to know how many people were using that service at the same time. It is not reasonable for us to ask users to report how many people were on at the same time as them, so the next best thing is an estimate. We can do this by taking the total passenger miles traveled (PMT) for each agency and then dividing that by the total miles of service the agency provided to get an average of how many passengers per mile they served. 

_Requires `pandas` and `openpyxl`_

In [173]:
ntd_service_pmt_data = {}

def refactor_service_pmt_data(url_entry):
    # Load in dataframe
    df = load_dataframe(url_entry, "service-pmt", "Annual Service Data By Mode")
    # Get rid of all NaN values
    df = df.fillna(0)
    # Convert all array rows into dictionaries
    converted_rows = [row.to_dict() for _, row in df.iterrows()]
    # Group all of the rows by NTD ID, and then mode that they pertain to
    aggregate_data = group_by_ntd_mode(converted_rows)
    # Add to overall data
    ntd_service_pmt_data[url_entry["year"]] = aggregate_data
    print("Service data year " + url_entry["year"] + " is finished!")
    
def group_by_ntd_mode(converted_rows):
    '''
    Group each row based on NTD id, and then group it once more based upon mode. Each mode must store an array
    of rows because can be multiple instances of same mode in same NTD id. 
    Example Structure:
    {
        "ntd_id": {
            "mode": [
                {
                    "field1": 1,
                    "field2": 2,
                    ...
                },
                {
                    "field1": 1,
                    "field2": 2,
                    ...
                }
            ]
        }
    }
    '''
    aggregate_data = defaultdict(lambda: defaultdict(list))
    for row in converted_rows:
        ntd_id = row["NTD ID"]
        mode = row["Mode"]
        aggregate_data[ntd_id][mode].append(row)
    return aggregate_data

for url_entry in ntd_service_pmt_urls:
    refactor_service_pmt_data(url_entry)

# Add all the data to a file
ntd_service_pmt_data_json = json.dumps(ntd_service_pmt_data, indent=2)
with open("ntd_service_pmt.json", 'w') as file:
    file.write(ntd_service_pmt_data_json)

# Delete all old files
delete_old_files("service-pmt", ntd_service_pmt_urls)

YEAR: 2022
Service data year 2022 is finished!
YEAR: 2021
Service data year 2021 is finished!
YEAR: 2020
Service data year 2020 is finished!
YEAR: 2019
Service data year 2019 is finished!
YEAR: 2018
Service data year 2018 is finished!
The file 2022-service-pmt.csv has been removed successfully.
The file 2021-service-pmt.xlsx has been removed successfully.
The file 2020-service-pmt.zip has been removed successfully.
The file 2019-service-pmt.zip has been removed successfully.
The file 2018-service-pmt.xlsm has been removed successfully.


### Carbon calculations

With all of our data reformatted, we can now calculate the kWh and kg of CO2 used per trip.

In [ ]:
# Import data
with open(combined_data_name, 'r') as file:
    combined_data = json.load(file)

# Find all key/column id's 
keys = [column["name"] for column in combined_data["meta"]["view"]["columns"]]

factors = {
    "Gasoline (gal)": {"kWh_per_unit": footprint_calculations.KWH_PER_GALLON_GASOLINE, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_GALLON_GASOLINE},
    "Diesel (gal)": {"kWh_per_unit": footprint_calculations.KWH_PER_GALLON_DIESEL, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_GALLON_DIESEL},
    "Bio-Diesel (gal)": {"kWh_per_unit": footprint_calculations.KWH_PER_GALLON_BIODIESEL, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_GALLON_BIODIESEL},
    "Liquefied Petroleum Gas (gal equivalent)": {"kWh_per_unit": footprint_calculations.KWH_PER_GALLON_LPG, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_GALLON_LPG},
    "Compressed Natural Gas (gal equivalent)": {"kWh_per_unit": footprint_calculations.KWH_PER_GALLON_CNG, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_GALLON_CNG},
    "Hydrogen (kg)": {"kWh_per_unit": footprint_calculations.KWH_PER_KG_HYDROGEN, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_KG_HYDROGEN},
    "Electric Propulsion (kWh)": {"kWh_per_unit": 1, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_KWH_ELECTRICITY},
    "Electric Battery (kWh)": {"kWh_per_unit": 1, "kg_CO2_per_unit": footprint_calculations.KG_CO2_PER_KWH_ELECTRICITY}
}

mode_conversion = {
    "Bus": ["CB", "MB", "RB", "TB"],
    "Train": ["LR", "CC", "SR", "TR", "CR", "HR", "MG", "YR"],
    "": []
}

def total_kwh_CO2(code, modes=[]):
    """
    Calculates total kWh and kg of CO2 within a certain UACE code and specified modes. It searches for all the units of
    energy listed above in factors, and then multiples it by a constant to calculate the values.
    :param code: UACE Code for a region
    :param modes: Array of modes that we are looking for
    """
    total_kWh = 0
    total_CO2 = 0
    mode_check = lambda row_mode: row_mode in modes
    for row in combined_data["data"][code]:
        if modes == [] or mode_check(row["Mode"]):
            for factor in factors:
                total_kWh += float(row[factor]) * factors[factor]["kWh_per_unit"]
                total_CO2 += float(row[factor]) * factors[factor]["kg_CO2_per_unit"]
    return (total_kWh, total_CO2)

def total_miles(code, modes=[]):
    """
    Finds the total amount of miles of public transportation used in a given UACE code with specified modes.
    :param code: UACE Code for a region
    :param modes: Array of modes that we are looking for
    """
    total = 0
    fuel_types = [k for k in keys if k.endswith("(miles)")]
    mode_check = lambda row_mode: row_mode in modes
    for row in combined_data["data"][code]:
        if modes == [] or mode_check(row["Mode"]):
            for fuel in fuel_types:
                total += float(row[fuel])
    return total

def calculate(code, trip_miles, mode, include_pmt=False):
    """
    Calculates total kWh and kg of CO2 used during a trip.
    :param code: UACE code trip took place in
    :param trip_miles: Number of miles the trip was
    :param include_pmt: Boolean to determine whether or not to include average load of public transit
    :param mode: Mode we want to search for 'Bus | Train'
    """
    # Convert e-mission mode to NTD modes
    mode_converted = mode_conversion[mode]

    # Total values
    (total_kWh, total_CO2) = total_kwh_CO2(code, mode_converted)
    miles = total_miles(code, mode_converted)

    # kWh and CO2 per mile
    kWh_mile = total_kWh / miles
    CO2_mile = total_CO2 / miles

    if include_pmt:
        # kWh and CO2 per passenger mile
        pmt = combined_data["pmt"][code]
        miles_pmt = total_miles(code)
        avg_passengers_per_mile = pmt / miles_pmt
        kWh_mile = kWh_mile / avg_passengers_per_mile
        CO2_mile = CO2_mile / avg_passengers_per_mile

    # kWh and CO2 for current trip
    kWh = kWh_mile * trip_miles
    CO2 = CO2_mile * trip_miles

    print("----------- Code: " + code + " | Mode: " + mode + " | PMT: " + str(include_pmt) + " -----------\n")
    print("Total kWh: " + str(total_kWh) + "\nTotal kg CO2: " + str(total_CO2) + "\nTotal Miles: " + str(miles) + "\n")
    print("kWh per mile: " + str(kWh_mile) + "\nkg CO2 per mile: " + str(CO2_mile) + "\n")
    if (include_pmt):
        print("Using PMT gives an average of " + str(avg_passengers_per_mile) + " passengers per mile\n")
    print("kWh for this trip: " + str(kWh) + "\nkg CO2 for this trip: " + str(CO2) + "\n")
    

### Running data analysis

In [ ]:
calculate("9271", 10, "Bus")

calculate("9271", 10, "Bus", include_pmt=True)

calculate("63217", 10, "Bus")

calculate("63217", 10, "Bus", include_pmt=True)
